### 3.2 Train Yolo OBB using Pytorch & Ultralytics Framework
- Setting GPU Environment
- Installing Ultralytics YOLO on Colab
- Download Dataset from Roboflow (previous notebook)
- Train Custom YOLO Detector
- Infer Custom Objects with Saved YOLO (`.pt`)

⚠️⚠️⚠️ *Please open this notebook in Google Colab* by click below link ⚠️⚠️⚠️<br><br>
<a href="https://colab.research.google.com/github/Muhammad-Yunus/OpenCV-ObjectAnalysis-Learn/blob/main/Pertemuan_3/3.2_train-yolo-obb-on-custom-dataset.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1.1 Connect GPU Environment

- Click `Connect` button in top right Google Colab notebook,<br>
<img src="https://github.com/Muhammad-Yunus/OpenCV-ObjectAnalysis-Learn/blob/main/Pertemuan_3/resource/cl-connect-gpu.png?raw=1" width="250px">
- If connecting process completed, it will turn to something look like this<br>
<img src="https://github.com/Muhammad-Yunus/OpenCV-ObjectAnalysis-Learn/blob/main/Pertemuan_3/resource/cl-connect-gpu-success.png?raw=1" width="250px">

- Check GPU connected into Colab environment is active

In [ ]:
!nvidia-smi

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

## 1.2 Installing Ultralytics YOLO on Colab
- `Ultralytics YOLO` is python library that help simplify YOLO training model in `Pytorch`
- It's wrap `Pytorch` functionality as simple CLI command like we use `Darknet` framework
- More about `Ultralytics YOLO` (https://docs.ultralytics.com/)  

In [ ]:
# Pip install method (recommended)

!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

In [ ]:
from ultralytics import YOLO

from IPython.display import display, Image

### 1.2.1 Ultralytics CLI Basics

If you want to train, validate or run inference on models and don't need to make any modifications to the code, using YOLO command line interface is the easiest way to get started. Read more about CLI in [Ultralytics YOLO Docs](https://docs.ultralytics.com/usage/cli/).

```
yolo task=detect    mode=train    model=yolov8s.yaml      args...
          classify       predict        yolov8s-cls.yaml  args...
          segment        val            yolov8s-seg.yaml  args...
          obb            export         yolov8s.pt        format=onnx  args...
```

## 1.3 Download Dataset from Roboflow (previous notebook)

- Back to `Roboflow` > `Project` > `Versions` menu
- Then click `Download Dataset`<br>
<img src="https://github.com/Muhammad-Yunus/OpenCV-ObjectAnalysis-Learn/blob/main/Pertemuan_3/resource/rb-download-dataset.png?raw=1" width="850px"><br><br>
- Choose `YOLO V8 Oriented Bounding Box` format and select `Show download code` then click `Continue` <br>
<img src="https://github.com/Muhammad-Yunus/OpenCV-ObjectAnalysis-Learn/blob/main/Pertemuan_3/resource/rb-download-format.png?raw=1" width="600px"><br><br>
- click `Copy` icon to copy roboflow download code<br>
<img src="https://github.com/Muhammad-Yunus/OpenCV-ObjectAnalysis-Learn/blob/main/Pertemuan_3/resource/rb-copy-download-code.png?raw=1" width="600px"><br><br>
- Then <font color="orange">replace below code</font> using the copied roboflow-download-code above,


In [ ]:
# !pip install roboflow

# from roboflow import Roboflow
# rf = Roboflow(api_key="xxxxxxxxxxxxxxxxxxxxxxxxx")
# project = rf.workspace("xxxxxxxxxxxxxxxxx").project("xxxxxxxxxxxxxxxxxxxxxxxxxxx")
# version = project.version(1)
# dataset = version.download("yolov8-obb")


## Custom Training
- Don't forget to change `imgsz=` to image size uploaded in Roboflow
- Default `imgsz=320`
- Default model is `yolov8s-obb.pt`

In [ ]:
%cd {HOME}

!yolo task=obb mode=train model=yolov8s-obb.pt data={dataset.location}/data.yaml epochs=25 imgsz=320 plots=True

In [ ]:
!ls {HOME}/runs/obb/train/

- Check Results Plot

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/obb/train/results.png', width=600)

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/obb/train/val_batch0_pred.jpg', width=600)

- Check mAP on Validation Dataset

In [ ]:
%cd {HOME}

!yolo task=obb mode=val model={HOME}/runs/obb/train/weights/best.pt data={dataset.location}/data.yaml

## Test inference on Custom Yolo V8 Model
- trained model located in `{HOME}/runs/detect/train/weights/best.pt`
- we will try to do inferencing using that model to detect test image in `{dataset.location}/test/images`

In [ ]:
%cd {HOME}
!yolo task=obb mode=predict model={HOME}/runs/obb/train/weights/best.pt conf=0.4 source={dataset.location}/test/images save=True

- Above command will generating detected image that located in `{HOME}/runs/detect/`
- Show the detection image result using script below

In [ ]:
import glob
from IPython.display import Image, display

# Define the base path where the folders are located
base_path = f'{HOME}/runs/obb/'

# List all directories that start with 'predict' in the base path
subfolders = [os.path.join(base_path, d) for d in os.listdir(base_path)
              if os.path.isdir(os.path.join(base_path, d)) and d.startswith('predict')]

# Find the latest folder by modification time
latest_folder = max(subfolders, key=os.path.getmtime)

image_paths = glob.glob(f'{latest_folder}/*.jpg')[:3]

# Display each image
for image_path in image_paths:
    display(Image(filename=image_path, width=320))
    print("\n")

## Download Trained Model 

In [ ]:
import shutil
from google.colab import files

shutil.copy(f"{HOME}/runs/obb/train/weights/best.pt",f"{HOME}/yolov8-obb-custom.pt")
files.download(f"{HOME}/yolov8-obb-custom.pt")

________________________________________________________________________

# Source
- https://blog.roboflow.com/train-yolov8-obb-model/
- https://github.com/roboflow/notebooks/blob/main/notebooks/train-yolov8-object-detection-on-custom-dataset.ipynb